Import necessary libraries

In [1]:
import numpy as py
import pandas as pd
import openpyxl 
from openpyxl.utils.dataframe import dataframe_to_rows

Load and Prep Data

In [2]:
#Load election results
election_results = pd.read_csv('Results.csv')
#Load new constituencies
file = 'New Constituencies.xlsx'
xl = pd.ExcelFile(file)
new_constituencies = xl.parse('Sheet9')
seats_df = xl.parse('Sheet1')
nc = pd.DataFrame(new_constituencies['New Constituencies'].unique().tolist())
nc_up = pd.merge(nc, seats_df, left_on=0,right_on = 'New Constituency')
nc_up=nc_up.drop([0,'Province'], axis = 1)

In [8]:
#Separate the election results into preliminary and validated 
#For this project, we will only be using validated election results
results_val = election_results.loc[election_results['Type of results'] == 'validated']
results_prelim = election_results.loc[election_results['Type of results'] == 'preliminary']

In [9]:
# Get aggregated number of votes for each political party
pei=results_val.groupby('Political affiliation')['Votes obtained'].sum()
pei=pei.to_frame()

In [12]:
#Merge the validated election results with the new_constituencies table. This essentially assigns newly created constituencies discussed in our research to vote based on the Provice from where the vote came.
df_combo = pd.merge(results_val,new_constituencies, left_on="Electoral district name",right_on ="Provinces")

In [14]:
#Drop columns that will not be used
df_combo=df_combo.drop(['Provinces','Appartenance politique','% Votes obtained'], axis = 1)
df_combo1 = df_combo.drop(['Type of results','Surname','Given name','Rejected ballots','Total number of ballots cast'], axis = 1)
df_combo1 = df_combo1.drop(["Middle name(s) "], axis = 1)

In [16]:
#Aggregate the votes obtained per party by each of the new constituencies that are discussed in the research
votesgrouped = df_combo1.groupby(['New Constituencies','Political affiliation'])["Votes obtained"].apply(lambda x : x.astype(int).sum())

In [76]:
votesgrouped.to_csv('Votes.csv',header=True,encoding='utf-8-sig')
votesgrouped = pd.read_csv('Votes.csv')

In [82]:
#Only select the parties that are classified as popular parties
array = ['Liberal','Conservative','Bloc Québécois','Green Party','NDP-New Democratic Party']
votesgrp1=votesgrouped.loc[votesgrouped['Political affiliation'].isin(array)]
#votesgrp1=votesgrouped

In [84]:
#pei_new=votesgrp1.groupby('Political affiliation')['Votes obtained'].sum()
pei_new=votesgrp1

In [86]:
#rename the original Votes Obtained column to Votes_Seat1
pei_new.rename(columns={'Votes obtained': 'Votes_Seat1'}, inplace=True)

In [89]:
df_merge = pd.merge(pei_new, nc_up, left_on='New Constituencies',right_on ='New Constituency')

In [91]:
df_merge.rename(columns={'New Constituencies': 'New_Constituencies'}, inplace=True)

In [106]:
df_merge=df_merge.drop(['Unnamed: 0','New Constituency'], axis = 1)

In this section we implement the D'Hondt formula. 
The requirement for this implementation is that there will be atleast one seat assignment. This implementation then allows the user to see the seat distributions between various parties based on any number of seat assignments.

In [111]:
wb = openpyxl.Workbook()

In [112]:
wb.save("DHondt Results.xlsx") 

In [113]:
for j in range(1,nc_up.shape[0]): #for each new constituency, we will calculate the seat allocation per party
    aa=df_merge[df_merge.New_Constituencies==nc_up['New Constituency'].iloc[j]]
    seats = int(nc_up['Number of Seats'].iloc[j])
    first_max_new = pei_new['Votes_Seat1'].max()
    seat_new = 0
    aa['Seats_Seat1'] = seat_new
    aa.reset_index(inplace=True)
    aa.loc[aa['Votes_Seat1'] == first_max_new, 'Seats_Seat1'] = 1
    for i in range (2,(seats+1)):
        col_name_votes = 'Votes_Seat'+str(i)
        col_name_votes_prev = 'Votes_Seat'+str(i-1)
        col_name_seats = 'Seats_Seat'+str(i)
        col_name_seats_prev = 'Seats_Seat'+str(i-1)
        aa[col_name_votes]=aa['Votes_Seat1']/(1+aa[col_name_seats_prev])
        max_val = aa[col_name_votes].max()
        aa[col_name_seats] = aa[col_name_seats_prev]
        aa.loc[aa[col_name_votes] == max_val, col_name_seats] = aa[col_name_seats_prev] + 1
    sheetname = nc_up['New Constituency'].iloc[j]
    sheet=wb.create_sheet(index = 1 , title = sheetname[:5]) 
    for r in dataframe_to_rows(aa, index=True, header=True):
        sheet.append(r)
wb.save("DHondt Results.xlsx") 
    

C:\Users\primu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\primu\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\primu\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa